In [ ]:
from transformers import pipeline

classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
def classify_response_zero_shot(input_text, candidate_labels=["positive", "negative", "neutral"]):
    result = classifier(input_text, candidate_labels)
    return result['labels'][0]  # Get the most probable label

/home/gerstenberger/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/gerstenberger/anaconda3/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/gerstenberger/anaconda3/lib/python3.11/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can al

In [ ]:
import transformers
import textwrap
from transformers import LlamaTokenizer, LlamaForCausalLM
import os
import sys
from typing import List

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_kbit_training
)

import fire
import torch
from datasets import load_dataset
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
from pylab import rcParams
import json

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

## Model

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel, PeftConfig
import torch

def load_model_and_tokenizer(model_name, state_dict_path=None, peft_adapter_dir=None):
    """
    Load the model and tokenizer, optionally load PEFT adapter and state dictionary.
    
    Parameters:
        model_name (str): The name of the model to load.
        state_dict_path (str, optional): The path to the state dictionary (if available).
        peft_adapter_dir (str, optional): Directory to load the PEFT adapter from.
    
    Returns:
        tuple: A tuple containing the model and tokenizer.
    """
    # Step 1: Load the base model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        torch_dtype=torch.float16,
        device_map="auto"  # Automatically map model to available GPUs
    )

    # Step 2: Load the state dictionary (if available)
    if state_dict_path:
        try:
            state_dict = torch.load(state_dict_path, weights_only=False, map_location="cuda" if torch.cuda.is_available() else "cpu")
            model.load_state_dict(state_dict, strict=False)
            print("State dictionary loaded successfully.")
        except Exception as e:
            print(f"Error loading state dictionary: {e}")

    # Step 3: Apply the PEFT adapter (if directory is provided)
    if peft_adapter_dir:
        try:
            peft_config = PeftConfig.from_pretrained(peft_adapter_dir)
            model = PeftModel.from_pretrained(model, peft_adapter_dir)
            print("PEFT adapter loaded successfully.")
        except Exception as e:
            print(f"Error loading PEFT adapter: {e}")

    # Step 4: Load the tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.pad_token_id = 0
    tokenizer.padding_side = "left"

    # Return the model and tokenizer
    return model, tokenizer

# Parameters
model_name = "RichardErkhov/meta-llama_-_Llama-3.2-3B-Instruct-8bits"
state_dict_path = "./experiments/checkpoint-50/rng_state.pth"
peft_adapter_dir = "./experiments"  # Set to None if you don't want to load PEFT adapter

# Load the model and tokenizer
model, tokenizer = load_model_and_tokenizer(
    model_name=model_name,
    state_dict_path=state_dict_path,
    peft_adapter_dir=peft_adapter_dir
)

/home/gerstenberger/anaconda3/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/home/gerstenberger/anaconda3/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(
/home/gerstenberger/anaconda3/lib/python3.11/site-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can al

Error loading state dictionary: [Errno 2] No such file or directory: './experiments/checkpoint-50/rng_state.pth'
Error loading PEFT adapter: Can't find 'adapter_config.json' at './experiments'


## Dataset

In [ ]:
from datasets import load_dataset
data_file_path = "./validation.json"
data = load_dataset("json", data_files=data_file_path)

In [ ]:
CUTOFF_LEN = 256
def generate_prompt(data_point):
    return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.  # noqa: E501
    ### Instruction:
    {data_point["instruction"]}
    ### Input:
    {data_point["input"]}
    ### Response:
    {data_point["output"]}"""

def tokenize(prompt, add_eos_token=True):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=CUTOFF_LEN,
        padding=False,
        return_tensors=None,
    )
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < CUTOFF_LEN
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    result["labels"] = result["input_ids"].copy()

    return result

def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenize(full_prompt)
    return tokenized_full_prompt

In [ ]:
train_val = data["train"].train_test_split(test_size=200, shuffle=True, seed=42)
train_data = (train_val["train"].shuffle().map(generate_and_tokenize_prompt))
val_data = (train_val["test"].shuffle().map(generate_and_tokenize_prompt))

In [15]:
def classify(item, model, tokenizer, classes = ["positive", "negative", "neutral", "irrelevant"], default_class = "irrelevant"):
    if not "input_ids" in item.keys():
        inputs = generate_and_tokenize_prompt(item)
    else:
        inputs = {key: item[key] for key in ['input_ids', 'labels', 'attention_mask']}
    inputs = {key: torch.tensor(val).unsqueeze(0).to(model.device) for key, val in inputs.items()}  # Add batch dimension and move to device
    outputs = model.generate(**inputs)
    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    text_answer = generated_text[generated_text.find("### Response:\n") +len("### Response:\n"):].lower()
    pred_class = "default_class"
    for i, c in enumerate(classes):
        if c.lower() in text_answer:
            pred_class = c.lower()
            break
    return text_answer, pred_class


print(classify({'instruction': 'Detect the sentiment of the tweet.', 'input': 'Bitcoin is hitting new all-time highs this year! #bullish #bitcoin', 'output': ''}, model, tokenizer))


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


('    \n\nthe sentiment of the tweet is bullish.', 'default_class')


In [16]:
y = []
y_hat = []
for item in val_data:
    y_hat.append(item["output"].lower())
    item["output"] = ""
    c = classify(item, model, tokenizer)[1]
    y.append(c)
    print(y_hat[-1])
    print(y[-1])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


neutral
neutral


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


positive
positive


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

neutral
neutral


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


neutral
neutral


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


positive
positive
negative
negative


In [24]:
acc = torch.mean(torch.tensor([a==b for a,b in zip(y, y_hat)], dtype=float))
print(acc)

tensor(0.9550, dtype=torch.float64)


In [3]:
import pickle

In [28]:
if False:
    with open("y.pkl", "wb") as f:
        pickle.dump(y, f)
    with open("y_hat.pkl", "wb") as f:
        pickle.dump(y_hat, f)

In [4]:
with open("y.pkl", "rb") as f:
    y = pickle.load(f)

with open("y_hat.pkl", "rb") as f:
    y_hat = pickle.load(f)

# Compute confusion matrix

In [45]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix

def compute_confusion_matrix(y_true, y_pred, labels):
    """
    Compute the confusion matrix for a classification problem and return it as a DataFrame.

    Args:
        y_true (list of str): Ground truth labels.
        y_pred (list of str): Predicted labels.
        labels (list of str): List of all possible class labels.

    Returns:
        pd.DataFrame: Confusion matrix with headers and index.
    """
    matrix = confusion_matrix(y_true, y_pred, labels=labels)
    return pd.DataFrame(matrix, index=[f"Actual: {label}" for label in labels], columns=[f"Predicted: {label}" for label in labels])

import pandas as pd
from sklearn.metrics import confusion_matrix

def compute_metrics(y_true, y_pred, labels):
    """
    Compute true positives, true negatives, false positives, and false negatives for each class.

    Args:
        y_true (list of str): Ground truth labels.
        y_pred (list of str): Predicted labels.
        labels (list of str): List of all possible class labels.

    Returns:
        pd.DataFrame: DataFrame containing TP, TN, FP, FN for each class.
    """
    matrix = confusion_matrix(y_true, y_pred, labels=labels)
    metrics = []

    for i, label in enumerate(labels):
        tp = matrix[i, i]
        fn = matrix[i, :].sum() - tp
        fp = matrix[:, i].sum() - tp
        tn = matrix.sum() - (tp + fn + fp)

        metrics.append({"Class": label, "TP": tp, "TN": tn, "FP": fp, "FN": fn})

    return pd.DataFrame(metrics)

def compute_sensitivity_specificity(y_true, y_pred, labels):
    """
    Compute sensitivity (TPR) and specificity (TNR) for each class.

    Args:
        y_true (list of str): Ground truth labels.
        y_pred (list of str): Predicted labels.
        labels (list of str): List of all possible class labels.

    Returns:
        pd.DataFrame: DataFrame containing TPR and TNR for each class.
    """
    matrix = confusion_matrix(y_true, y_pred, labels=labels)
    metrics = []

    for i, label in enumerate(labels):
        tp = matrix[i, i]
        fn = matrix[i, :].sum() - tp
        fp = matrix[:, i].sum() - tp
        tn = matrix.sum() - (tp + fn + fp)

        tpr = tp / (tp + fn) if (tp + fn) > 0 else 0  # Sensitivity: TP / (TP + FN)
        tnr = tn / (tn + fp) if (tn + fp) > 0 else 0  # Specificity: TN / (TN + FP)

        metrics.append({"Class": label, "Sensitivity (TPR)": tpr, "Specificity (TNR)": tnr})

    return pd.DataFrame(metrics)


In [46]:
compute_confusion_matrix(y, y_hat, ["positive", "negative", "neutral", "irrelevant"])

Predicted: positive  Predicted: negative  \
Actual: positive                     51                    1   
Actual: negative                      0                   45   
Actual: neutral                       0                    0   
Actual: irrelevant                    0                    0   

                    Predicted: neutral  Predicted: irrelevant  
Actual: positive                     0                      2  
Actual: negative                     0                      4  
Actual: neutral                     64                      2  
Actual: irrelevant                   0                     31

In [47]:
compute_metrics(y, y_hat, ["positive", "negative", "neutral", "irrelevant"])

Class  TP   TN  FP  FN
0    positive  51  146   0   3
1    negative  45  150   1   4
2     neutral  64  134   0   2
3  irrelevant  31  161   8   0

In [51]:
compute_sensitivity_specificity(y, y_hat, ["positive", "negative", "neutral", "irrelevant"])

Class  Sensitivity (TPR)  Specificity (TNR)
0    positive           0.944444           1.000000
1    negative           0.918367           0.993377
2     neutral           0.969697           1.000000
3  irrelevant           1.000000           0.952663